In [1]:
import pandas as pd
import json
from pathlib import Path
import sys
from PIL import Image, ImageDraw
sys.path.append("../")
from utils import read_meta, MetaData, KeypointInfo

In [2]:
def json_to_df(path:Path)->pd.DataFrame:
    cols = ["image_path", "frame_no", "model_name"]
    # cols.extend([o for o in KeypointInfo._fields])
    cols.extend([f"kp_{i}" for i in range(12)])
    df =  pd.DataFrame(columns=cols)

    for i, fn in enumerate(path.glob("*.json")):
        meta_data = read_meta(fn)
        image_path = meta_data.source["image"]
        model_name = meta_data.source["object_id"]
        frame_no = int(image_path.split("/")[-1].split("_")[-1].split(".")[0])
        d = [image_path, frame_no, model_name]
        d.extend(meta_data.keypoints)
        df.loc[i] = d
        # print(meta_data)
        # break
    return df


In [13]:
input_data_name="IMG_1094"
ref_data_path= Path(f"../results/{input_data_name}/annotated/ref/")
test_data_path = Path(f"../results/{input_data_name}/annotated/test/")
df_ref = json_to_df(ref_data_path)
df_test = json_to_df(test_data_path)

df_ref = df_ref.add_prefix("ref_")
df_test = df_test.add_prefix("test_")
df_comb = pd.concat([df_ref,df_test], axis=1)

ref_model_name = df_comb["ref_model_name"].unique()[0]
test_model_name = df_comb["test_model_name"].unique()[0].split("/")[-1]


dst_dir = f"../comparisons/{input_data_name}/{test_model_name}"
Path(dst_dir).mkdir(parents=True, exist_ok=True)

assert all(df_comb[["ref_frame_no", "test_frame_no"]].apply(lambda x: x[0]==x[1], axis=1))
assert len(df_comb["ref_model_name"].unique()) == 1
assert len(df_comb["ref_model_name"].unique()) == 1

/Users/benjamin/.pyenv/versions/3.8.6/envs/ml38/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray



In [14]:
test_model_name.split("/")[-1]

'3773fy6g - 2021-10-10T12:06:19.844863'

In [15]:
from functools import partial
def pixel_distance(x_ratio, y_ratio, p1, p2)->float:
    return ((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)**0.5

w,h = Image.open(f"../{df_comb['ref_image_path'].loc[0]}").size
pixel_distance = partial(pixel_distance, x_ratio = (160/w), y_ratio=(160/h))
for i in range(12):
    df_comb[f"diff_kp_{i}"] = df_comb.apply(lambda x: pixel_distance(p1=x[f"ref_kp_{i}"],p2=x[f"test_kp_{i}"]), axis=1)

In [16]:
pd.options.plotting.backend = "plotly"
cols=[]
for i in range(12):
    cols.append(f"diff_kp_{i}")
df_comb[cols].plot()
fig = df_comb[cols].describe().loc[["mean", "min", "max", "75%"]].plot(kind="bar", barmode='group', title=f"{input_data_name} - {ref_model_name} {test_model_name}")
fig.write_html(f"{dst_dir}/stats.html")

In [17]:
# fig.add_annotation()


In [18]:
fig = df_comb[cols].plot(title=f"{input_data_name} - {ref_model_name} {test_model_name}")
fig.write_html(f"{dst_dir}/pixel_errors.html")
dst_dir

'../comparisons/IMG_1094/3773fy6g - 2021-10-10T12:06:19.844863'

In [19]:
worst_frames={}
worst_n=10
for c in cols:
    info = df_comb[c].nlargest(worst_n).index.tolist()
    errors = df_comb[c].nlargest(worst_n).tolist()

    worst_frames[c]= list(zip(info, errors))


In [20]:
    
for k,v in worst_frames.items():
    print(f"{k}: {v}")
    worsts_image = Image.new("RGB", (len(v)*160, 160*2))

    x_offset = 0
    for i, error in v:
        print(df_comb['ref_image_path'].loc[i])
        test_image_path = df_comb['ref_image_path'].loc[i].split("/")
        test_image_path.insert(-1, 'annotated/test')
        annotated_image_path = "/".join(test_image_path)
        wi = Image.open(f"../{annotated_image_path}")
        w, h = wi.size
        wi = wi.resize((160,160))
        draw = ImageDraw.Draw(wi)
        draw.text((20,20),"Bouldrama", (255,255,255))
        draw.text((20,30),f"error: {error:.2f}", (255,255,255))
        x, y = df_comb[k.replace("diff", "test")].loc[i]
        x, y = (x/w)*160, (y/h)*160
        # draw.rectangle((10, 10, 100, 100), fill ="#ffff33", outline ="red")
        pointer_size=2
        draw.rectangle((x-pointer_size, y-pointer_size, x+pointer_size, y+pointer_size), fill ="#ffff33", outline ="red")

        worsts_image.paste(
            wi,
            (x_offset, 0),
        )
        x_offset += 160
    x_offset = 0
    for i, error in v:
        test_image_path = df_comb['ref_image_path'].loc[i].split("/")
        test_image_path.insert(-1, 'annotated/ref')
        annotated_image_path = "/".join(test_image_path)
        wi = Image.open(f"../{annotated_image_path}")
        w, h = wi.size
        wi = wi.resize((160,160))
        draw = ImageDraw.Draw(wi)
        draw.text((20,20),"Ref", (255,255,255))
        draw.text((20,30),f"error: {error:.2f}", (255,255,255))
        x, y = df_comb[k.replace("diff", "ref")].loc[i]
        x, y = (x/w)*160, (y/h)*160
        # draw.rectangle((10, 10, 100, 100), fill ="#ffff33", outline ="red")
        pointer_size=2
        draw.rectangle((x-pointer_size, y-pointer_size, x+pointer_size, y+pointer_size), fill ="#ffff33", outline ="red")
        worsts_image.paste(
            wi,
            (x_offset, 160),
        )
        x_offset += 160
    worsts_image.save(f"{dst_dir}/worst_{worst_n}_pixel_{k}.png")

diff_kp_0: [(73, 27.730849247724095), (273, 26.627053911388696), (11, 25.059928172283335), (299, 24.413111231467404), (270, 24.186773244895647), (2, 23.323807579381203), (180, 22.825424421026653), (177, 16.0), (287, 16.0), (150, 15.231546211727817)]
results/IMG_1094/frame_187.png
results/IMG_1094/frame_189.png
results/IMG_1094/frame_190.png
results/IMG_1094/frame_193.png
results/IMG_1094/frame_188.png
results/IMG_1094/frame_186.png
results/IMG_1094/frame_194.png
results/IMG_1094/frame_045.png
results/IMG_1094/frame_058.png
results/IMG_1094/frame_162.png
diff_kp_1: [(207, 104.5609869884557), (175, 104.10091258005379), (140, 98.79271228182775), (68, 57.48912940721924), (269, 27.0), (200, 20.248456731316587), (246, 19.849433241279208), (298, 19.79898987322333), (74, 19.026297590440446), (9, 18.439088914585774)]
results/IMG_1094/frame_156.png
results/IMG_1094/frame_157.png
results/IMG_1094/frame_158.png
results/IMG_1094/frame_129.png
results/IMG_1094/frame_167.png
results/IMG_1094/frame_25

In [21]:
# import subprocess
# import shlex
# import os
# cmd = f"ffmpeg -framerate 25 -i \'{os.fsdecode(ref_data_path.absolute())}/frame_%d.png\' -c:v libx264 -pix_fmt yuv420p ref.mov"
# subprocess.check_call(shlex.split(cmd))
import os
print(f"ffmpeg -framerate 25 -i \"/Users/benjamin/projects/keypoint-tracking/results/{input_data_name}/annotated/ref/frame_%3d.png\" -c:v libx264 -pix_fmt yuv420p \"/Users/benjamin/projects/keypoint-tracking/comparisons/{input_data_name}/{Path(dst_dir).stem}{Path(dst_dir).suffix}/ref.mov\"")
print(f"ffmpeg -framerate 25 -i \"/Users/benjamin/projects/keypoint-tracking/results/{input_data_name}/annotated/test/frame_%3d.png\" -c:v libx264 -pix_fmt yuv420p \"/Users/benjamin/projects/keypoint-tracking/comparisons/{input_data_name}/{Path(dst_dir).stem}{Path(dst_dir).suffix}/test.mov\"")


ffmpeg -framerate 25 -i "/Users/benjamin/projects/keypoint-tracking/results/IMG_1094/annotated/ref/frame_%3d.png" -c:v libx264 -pix_fmt yuv420p "/Users/benjamin/projects/keypoint-tracking/comparisons/IMG_1094/3773fy6g - 2021-10-10T12:06:19.844863/ref.mov"
ffmpeg -framerate 25 -i "/Users/benjamin/projects/keypoint-tracking/results/IMG_1094/annotated/test/frame_%3d.png" -c:v libx264 -pix_fmt yuv420p "/Users/benjamin/projects/keypoint-tracking/comparisons/IMG_1094/3773fy6g - 2021-10-10T12:06:19.844863/test.mov"
